# Pygcdl Sample Data Creation

In this notebook, we create a set of spatial geometries for pygcdl testing purposes. These geometries are based on california county boundaries. For each polygon/multipolygon geometry, we also calculate the union / convex hull ratio, as this value is calculated by the pygcdl package when determining how to upload a geometry object.

Description of geometry files created in this notebook: </br>
<i>(Note: zip files contain shapefile files)</i>
- <b>yolo_county.zip:</b> Zipped shapefile of Yolo county boundaries
- <b>yolo_county.shp:</b> Non-zipped shapefile of Yolo county boundaries
- <b>loney_polygon/yolo_county.shp:</b> Same as above, but without associated files. Only .shp file is present in the lonely_polygon directory. This is to test that pygcdl will generate an error if associated files are not found.
- <b>yolo_county.geojson:</b> Geojson version of Yolo county boundaries
- <b>yolo_sac_counties.shp:</b> Non-zipped shapefile of Yolo and Sacramento county boundaries, adjacent polygons
- <b>ventura_county.zip: </b> Zipped shapefile of Ventura county, multipolygon data because of islands
- <b>county_centroids.zip:</b> Zipped shapefile of all CA county centroids
- <b>county_centroids.csv:</b> CSV version of county_centroids.shp

# Part 0: Prepare CA Counties Dataset

In [ ]:
import geopandas as gpd
from pathlib import Path
import zipfile
import matplotlib.pyplot as plt
import pandas as pd
import shapely
from shapely.geometry import MultiPolygon
import urllib.request
pd.options.mode.chained_assignment = None  # default='warn'
import shutil

In [ ]:
# Download and unzip CA county boundaries data
download_dir = Path("ca_county_data")
download_dir.mkdir(exist_ok=True)
download_file = Path("ca_county_data/ca_counties.zip")
urllib.request.urlretrieve("https://data.ca.gov/dataset/e212e397-1277-4df3-8c22-40721b095f33/resource/b0007416-a325-4777-9295-368ea6b710e6/download/ca_counties.zip", download_file)
with zipfile.ZipFile(download_file, 'r') as z:
    z.extractall(path=download_dir)
counties_shp_path = Path("ca_county_data/ca_counties.shp")

In [ ]:
# Define output directory to store sample geometries
output_dir = Path.cwd() / "sample_geoms"
output_dir.mkdir(exist_ok=True)

In [ ]:
# Define paths and read in county data
counties = gpd.read_file(counties_shp_path)
counties.head()

In [ ]:
# Remove all unnecessary columns
counties = counties[["NAME", "geometry"]]
counties.head()

In [ ]:
# Visualize all counties with county names
counties["centroid"] = counties["geometry"].centroid
fig, ax = plt.subplots(figsize=(20, 16))
counties.boundary.plot(ax=ax)
for idx, row in counties.iterrows():
    plt.annotate(text=row["NAME"], xy=(row["centroid"].x, row["centroid"].y), horizontalalignment='center', color='blue')
counties = counties.drop(["centroid"], axis=1) # remove row when done using it

In [ ]:
# We see here that our CRS is projected and in units of meters
counties.crs

As our last preparatory step, we define a function that will calculate the polygon union / convex hull ratio.

In [ ]:
def union_convex_hull_ratio(gdf = gpd.GeoDataFrame):
    union_polygon = gdf.unary_union
    union_area = union_polygon.area
    convex_hull_area = union_polygon.convex_hull.area
    ratio = round(union_area / convex_hull_area, ndigits=3)
    return ratio

# Part 1: Creating county polygon subsets

In [ ]:
# Create geodataframe with Yolo county
yolo_county = counties[counties["NAME"] == "Yolo"]
yolo_county.boundary.plot()
print("Union / Convex Hull ratio: ", union_convex_hull_ratio(yolo_county))

In [ ]:
# Store Yolo county to a zipped shapefile, single_polygon.zip

# Create folder to store shapefile files in
zip_dir = Path(output_dir/"yolo_county")
zip_dir.mkdir(exist_ok=True)

# Put shapefiles in folder
yolo_county.to_file(zip_dir/"yolo_county.shp")

# Zip folder
with zipfile.ZipFile(zip_dir.with_suffix(".zip"), 'w') as z:
    for filepath in zip_dir.glob("*.*"):
        z.write(filepath, arcname=filepath.name)

# Delete now unnecessary folder
shutil.rmtree(zip_dir)

# List files in newly created archive
with zipfile.ZipFile(zip_dir.with_suffix(".zip"), "r") as z:
    z.printdir()

In [ ]:
# Store Yolo county data to a shapefile, not in a .zip file
yolo_county.to_file(output_dir/"yolo_county.shp")

In [ ]:
# Store Yolo county data to a geojson file
yolo_county.to_file(output_dir / "yolo_county.geojson", driver="GeoJSON")

In [ ]:
# Create a folder for just a .shp file and no associated files
# Copy yolo_county.shp over to "lonely_shp" directory
# This is to test that pygcdl throws an error if associated files
# are not found in same directory as .shp file
lonely_shp_dir = Path(output_dir/"lonely_shp")
lonely_shp_dir.mkdir(exist_ok=True)
lonely_polygon_path = Path(lonely_shp_dir / "yolo_county.shp")
shutil.copyfile(Path(output_dir/"yolo_county.shp"), lonely_polygon_path)

In [ ]:
# Create geodataframe of two adjacent counties, Yolo and Sacramento
two_counties = ["Yolo", "Sacramento"]
yolo_sac_counties = counties[counties["NAME"].isin(two_counties)]
yolo_sac_counties.boundary.plot()
print("Union / Convex Hull ratio: ", union_convex_hull_ratio(yolo_sac_counties))

In [ ]:
# Save Yolo and Sac county data to a zipped shapefile, yolo_sac.zip

# Create folder to store shapefile files in
zip_dir = Path(output_dir/"yolo_sac_counties")
zip_dir.mkdir(exist_ok=True)

# Put shapefiles in folder
yolo_sac_counties.to_file(zip_dir/"yolo_sac_counties.shp")

# Zip folder
with zipfile.ZipFile(zip_dir.with_suffix(".zip"), 'w') as z:
    for filepath in zip_dir.glob("*.*"):
        z.write(filepath, arcname=filepath.name)

# Delete now unnecessary folder
shutil.rmtree(zip_dir)

# List files in newly created archive
with zipfile.ZipFile(zip_dir.with_suffix(".zip"), "r") as z:
    z.printdir()

In [ ]:
# Create a geodataframe of one county with islands
ventura_county = counties[counties["NAME"] == "Ventura"]
ventura_county.boundary.plot()
print("Union / Convex Hull ratio: ", union_convex_hull_ratio(ventura_county))

In [ ]:
# Save Ventura county data to a zipped shapefile, yolo_sac.zip

# Create folder to store shapefile files in
zip_dir = Path(output_dir/"ventura_county")
zip_dir.mkdir(exist_ok=True)

# Put shapefiles in folder
ventura_county.to_file(zip_dir/"ventura_county.shp")

# Zip folder
with zipfile.ZipFile(zip_dir.with_suffix(".zip"), 'w') as z:
    for filepath in zip_dir.glob("*.*"):
        z.write(filepath, arcname=filepath.name)

# Delete now unnecessary folder
shutil.rmtree(zip_dir)

# List files in newly created archive
with zipfile.ZipFile(zip_dir.with_suffix(".zip"), "r") as z:
    z.printdir()

# Part 2: Creating Point Data

##### Make shapefile of points

In [ ]:
# Create a geodataframe of the centroids of each county
counties["centroid"] = counties["geometry"].centroid
county_centroids = counties.set_geometry("centroid")
county_centroids = county_centroids.drop(columns="geometry")
county_centroids.plot()

In [ ]:
# Save points data as a zipped shapefile

# Create folder to store shapefile files in
zip_dir = Path(output_dir/"county_centroids")
zip_dir.mkdir(exist_ok=True)

# Put shapefiles in folder
county_centroids.to_file(zip_dir/"county_centroids.shp")

# Zip folder
with zipfile.ZipFile(zip_dir.with_suffix(".zip"), 'w') as z:
    for filepath in zip_dir.glob("*.*"):
        z.write(filepath, arcname=filepath.name)

# Delete now unnecessary folder
shutil.rmtree(zip_dir)

# List files in newly created archive
with zipfile.ZipFile(zip_dir.with_suffix(".zip"), "r") as z:
    z.printdir()

Next, we will store our point data into a csv file. CSV data doesn't contain geographic metadata, so when we request data downloads, we will need to specify our CRS so GCDL knows how to interpret the values in the CSV file. Previously, we saw that our counties data is in EPSG:3857. We will take note of this, and use this CRS when we request point data downloads using this CSV geometry data.

In [ ]:
# Extract x and y values of points into lists
x_values = [p.x for p in county_centroids["centroid"]]
y_values = [p.y for p in county_centroids["centroid"]]

In [ ]:
# Create pandas dataframe from x and y value lists
point_data = pd.DataFrame({"x":x_values, "y":y_values})
point_data.head()

In [ ]:
# Save x and y coordinates of point data as csv
point_data.to_csv(output_dir / "county_centroids.csv")